# 데이터 생성

In [1]:
import sqlite3
import pandas as pd

# CSV 파일 경로
customer_csv = "./database/customers.csv"
product_csv = "./database/products.csv"
category_csv = "./database/categories.csv"
employee_csv = "./database/employees.csv"
shipper_csv = "./database/shippers.csv"
supplier_csv = "./database/suppliers.csv"
order_csv = "./database/orders.csv"
orderdetail_csv = "./database/order_details.csv"

# CSV 로드
customers_df = pd.read_csv(customer_csv)
products_df = pd.read_csv(product_csv)
categories_df = pd.read_csv(category_csv)
employees_df = pd.read_csv(employee_csv)
shipper_df = pd.read_csv(shipper_csv)
suppliers_df = pd.read_csv(supplier_csv)
order_df = pd.read_csv(order_csv)
orderdetails_df = pd.read_csv(orderdetail_csv)

# SQLite 연결
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# 테이블 생성

# Customers
cur.execute("""
CREATE TABLE Customers (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName TEXT NOT NULL,
    ContactName TEXT,
    Address TEXT,
    City TEXT,
    PostalCode TEXT,
    Country TEXT
);
""")

# Products
cur.execute("""
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT NOT NULL,
    SupplierID INTEGER,
    CategoryID INTEGER,
    Unit TEXT,
    Price REAL,
    FOREIGN KEY (SupplierID) REFERENCES Suppliers(SupplierID),
    FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)
);
""")

# Categories
cur.execute("""
CREATE TABLE Categories (
    CategoryID INTEGER PRIMARY KEY,
    CategoryName TEXT NOT NULL,
    Description TEXT
);
""")

# Employees
cur.execute("""
CREATE TABLE Employees (
    EmployeeID INTEGER PRIMARY KEY,
    LastName TEXT,
    FirstName TEXT,
    BirthDate TEXT,
    Photo TEXT,
    Notes TEXT
);
""")

# Shippers
cur.execute("""
CREATE TABLE Shippers (
    ShipperID INTEGER PRIMARY KEY,
    ShipperName TEXT NOT NULL,
    Phone TEXT
);
""")

# Suppliers
cur.execute("""
CREATE TABLE Suppliers (
    SupplierID INTEGER PRIMARY KEY,
    SupplierName TEXT NOT NULL,
    ContactName TEXT,
    Address TEXT,
    City TEXT,
    PostalCode TEXT,
    Country TEXT,
    Phone TEXT
);
""")

# Orders
cur.execute("""
CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER NOT NULL,
    EmployeeID INTEGER NOT NULL,
    OrderDate TEXT NOT NULL,
    ShipperID INTEGER NOT NULL,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID),
    FOREIGN KEY (ShipperID) REFERENCES Shippers(ShipperID)
);
""")

# OrderDetails
cur.execute("""
CREATE TABLE OrderDetails (
    OrderDetailID INTEGER PRIMARY KEY,
    OrderID INTEGER NOT NULL,
    ProductID INTEGER NOT NULL,
    Quantity INTEGER NOT NULL,
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
""")

# 데이터 삽입
customers_df.to_sql("Customers", conn, if_exists="append", index=False)
products_df.to_sql("Products", conn, if_exists="append", index=False)
categories_df.to_sql("Categories", conn, if_exists="append", index=False)
employees_df.to_sql("Employees", conn, if_exists="append", index=False)
suppliers_df.to_sql("Suppliers", conn, if_exists="append", index=False)
shipper_df.to_sql("Shippers", conn, if_exists="append", index=False)
order_df.to_sql("Orders", conn, if_exists="append", index=False)
orderdetails_df.to_sql("OrderDetails", conn, if_exists="append", index=False)

518

In [2]:
def execute(query, print_limit=5):
    print("-" * 20)
    print(f"QUERY: {query}")
    print(f"RESULT:")
    for i, row in enumerate(cur.execute(query)):
        if i < print_limit:
            print(row)
        else:
            break
    print()
        
def commit():
    conn.commit()

## 1. SELECT, FROM, WHERE

In [3]:
query ="""
SELECT * FROM Customers;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers;

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [4]:
query ="""
SELECT CustomerName, City FROM Customers;
"""
execute(query)

--------------------
QUERY: 
SELECT CustomerName, City FROM Customers;

RESULT:
('Alfreds Futterkiste ', 'Berlin ')
('Ana Trujillo Emparedados y helados ', 'México D.F. ')
('Antonio Moreno Taquería ', 'México D.F. ')
('Around the Horn ', 'London ')
('Berglunds snabbköp ', 'Luleå ')



In [5]:
query ="""
SELECT DISTINCT Country FROM Customers;
"""
execute(query)

--------------------
QUERY: 
SELECT DISTINCT Country FROM Customers;

RESULT:
('Germany',)
('Mexico',)
('UK',)
('Sweden',)
('France',)



In [6]:
query ="""
SELECT COUNT(DISTINCT Country) FROM Customers;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(DISTINCT Country) FROM Customers;

RESULT:
(21,)



In [7]:
query ="""
SELECT Count(*) AS DistinctCountries
FROM (SELECT DISTINCT Country FROM Customers);
"""
execute(query)

--------------------
QUERY: 
SELECT Count(*) AS DistinctCountries
FROM (SELECT DISTINCT Country FROM Customers);

RESULT:
(21,)



In [8]:
query ="""
SELECT * FROM Customers
WHERE Country='Mexico';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country='Mexico';

RESULT:
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(13, 'Centro comercial Moctezuma ', 'Francisco Chang ', 'Sierras de Granada 9993 ', 'México D.F. ', '05022 ', 'Mexico')
(58, 'Pericles Comidas clásicas ', 'Guillermo Fernández ', 'Calle Dr. Jorge Cash 321 ', 'México D.F. ', '05033 ', 'Mexico')
(80, 'Tortuga Restaurante ', 'Miguel Angel Paolino ', 'Avda. Azteca 123 ', 'México D.F. ', '05033 ', 'Mexico')



In [9]:
query ="""
SELECT * FROM Customers
WHERE CustomerID=1;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerID=1;

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')



In [10]:
query ="""
SELECT * FROM Customers
WHERE CustomerID > 80;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerID > 80;

RESULT:
(81, 'Tradição Hipermercados ', 'Anabela Domingues ', 'Av. Inês de Castro, 414 ', 'São Paulo ', '05634-030 ', 'Brazil')
(82, "Trail's Head Gourmet Provisioners ", 'Helvetius Nagy ', '722 DaVinci Blvd. ', 'Kirkland ', '98034 ', 'USA')
(83, 'Vaffeljernet ', 'Palle Ibsen ', 'Smagsløget 45 ', 'Århus ', '8200 ', 'Denmark')
(84, 'Victuailles en stock ', 'Mary Saveley ', '2, rue du Commerce ', 'Lyon ', '69004 ', 'France')
(85, 'Vins et alcools Chevalier ', 'Paul Henriot ', "59 rue de l'Abbaye ", 'Reims ', '51100 ', 'France')



In [11]:
query ="""
SELECT * FROM Products
ORDER BY Price;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
ORDER BY Price;

RESULT:
(33, 'Geitost', 15, 4, '500 g', 2.5)
(24, 'Guaraná Fantástica', 10, 1, '12 - 355 ml cans', 4.5)
(13, 'Konbu', 6, 8, '2 kg box', 6.0)
(52, 'Filo Mix', 24, 5, '16 - 2 kg boxes', 7.0)
(54, 'Tourtière', 25, 6, '16 pies', 7.45)



In [12]:
query ="""
SELECT * FROM Products
ORDER BY Price DESC;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
ORDER BY Price DESC;

RESULT:
(38, 'Côte de Blaye', 18, 1, '12 - 75 cl bottles', 263.5)
(29, 'Thüringer Rostbratwurst', 12, 6, '50 bags x 30 sausgs.', 123.79)
(9, 'Mishi Kobe Niku', 4, 6, '18 - 500 g pkgs.', 97.0)
(20, "Sir Rodney's Marmalade", 8, 3, '30 gift boxes', 81.0)
(18, 'Carnarvon Tigers', 7, 8, '16 kg pkg.', 62.5)



In [13]:
query ="""
SELECT * FROM Products
ORDER BY ProductName;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
ORDER BY ProductName;

RESULT:
(17, 'Alice Mutton', 7, 6, '20 - 1 kg tins', 39.0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0)
(40, 'Boston Crab Meat', 19, 8, '24 - 4 oz tins', 18.4)
(60, 'Camembert Pierrot', 28, 4, '15 - 300 g rounds', 34.0)
(18, 'Carnarvon Tigers', 7, 8, '16 kg pkg.', 62.5)



In [14]:
query ="""
SELECT * FROM Products
ORDER BY ProductName DESC;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
ORDER BY ProductName DESC;

RESULT:
(47, 'Zaanse koeken', 22, 3, '10 - 4 oz boxes', 9.5)
(64, 'Wimmers gute Semmelknödel', 12, 5, '20 bags x 4 pieces', 33.25)
(63, 'Vegie-spread', 7, 2, '15 - 625 g jars', 43.9)
(50, 'Valkoinen suklaa', 23, 3, '12 - 100 g bars', 16.25)
(7, "Uncle Bob's Organic Dried Pears", 3, 7, '12 - 1 lb pkgs.', 30.0)



In [15]:
query ="""
SELECT * FROM Customers
ORDER BY Country, CustomerName;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
ORDER BY Country, CustomerName;

RESULT:
(12, 'Cactus Comidas para llevar ', 'Patricio Simpson ', 'Cerrito 333 ', 'Buenos Aires ', '1010 ', 'Argentina')
(54, 'Océano Atlántico Ltda. ', 'Yvonne Moncada ', 'Ing. Gustavo Moncada 8585 Piso 20-A ', 'Buenos Aires ', '1010 ', 'Argentina')
(64, 'Rancho grande ', 'Sergio Gutiérrez ', 'Av. del Libertador 900 ', 'Buenos Aires ', '1010 ', 'Argentina')
(20, 'Ernst Handel ', 'Roland Mendel ', 'Kirchgasse 6 ', 'Graz ', '8010 ', 'Austria')
(59, 'Piccolo und mehr ', 'Georg Pipps ', 'Geislweg 14 ', 'Salzburg ', '5020 ', 'Austria')



In [16]:
query ="""
SELECT * FROM Customers
ORDER BY Country ASC, CustomerName DESC;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
ORDER BY Country ASC, CustomerName DESC;

RESULT:
(64, 'Rancho grande ', 'Sergio Gutiérrez ', 'Av. del Libertador 900 ', 'Buenos Aires ', '1010 ', 'Argentina')
(54, 'Océano Atlántico Ltda. ', 'Yvonne Moncada ', 'Ing. Gustavo Moncada 8585 Piso 20-A ', 'Buenos Aires ', '1010 ', 'Argentina')
(12, 'Cactus Comidas para llevar ', 'Patricio Simpson ', 'Cerrito 333 ', 'Buenos Aires ', '1010 ', 'Argentina')
(59, 'Piccolo und mehr ', 'Georg Pipps ', 'Geislweg 14 ', 'Salzburg ', '5020 ', 'Austria')
(20, 'Ernst Handel ', 'Roland Mendel ', 'Kirchgasse 6 ', 'Graz ', '8010 ', 'Austria')



## 2. AND, OR, NOT

In [17]:
query ="""
SELECT *
FROM Customers
WHERE Country = 'Spain' AND CustomerName LIKE 'G%';
"""
execute(query)

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE Country = 'Spain' AND CustomerName LIKE 'G%';

RESULT:
(29, 'Galería del gastrónomo ', 'Eduardo Saavedra ', 'Rambla de Cataluña, 23 ', 'Barcelona ', '08022 ', 'Spain')
(30, 'Godos Cocina Típica ', 'José Pedro Freyre ', 'C/ Romero, 33 ', 'Sevilla ', '41101 ', 'Spain')



In [18]:
query ="""
SELECT * FROM Customers
WHERE Country = 'Brazil'
AND City = 'Rio de Janeiro'
AND CustomerID > 50;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country = 'Brazil'
AND City = 'Rio de Janeiro'
AND CustomerID > 50;

RESULT:



In [19]:
query ="""
SELECT * FROM Customers
WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');

RESULT:
(29, 'Galería del gastrónomo ', 'Eduardo Saavedra ', 'Rambla de Cataluña, 23 ', 'Barcelona ', '08022 ', 'Spain')
(30, 'Godos Cocina Típica ', 'José Pedro Freyre ', 'C/ Romero, 33 ', 'Sevilla ', '41101 ', 'Spain')
(69, 'Romero y tomillo ', 'Alejandra Camino ', 'Gran Vía, 1 ', 'Madrid ', '28001 ', 'Spain')



In [20]:
query ="""
SELECT *
FROM Customers
WHERE Country = 'Germany' OR Country = 'Spain';
"""
execute(query)

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE Country = 'Germany' OR Country = 'Spain';

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(6, 'Blauer See Delikatessen ', 'Hanna Moos ', 'Forsterstr. 57 ', 'Mannheim ', '68306 ', 'Germany')
(8, 'Bólido Comidas preparadas ', 'Martín Sommer ', 'C/ Araquil, 67 ', 'Madrid ', '28023 ', 'Spain')
(17, 'Drachenblut Delikatessend ', 'Sven Ottlieb ', 'Walserweg 21 ', 'Aachen ', '52066 ', 'Germany')
(22, 'FISSA Fabrica Inter. Salchichas S.A. ', 'Diego Roel ', 'C/ Moralzarzal, 86 ', 'Madrid ', '28034 ', 'Spain')



In [21]:
query ="""
SELECT * FROM Customers
WHERE City = 'Berlin' OR CustomerName LIKE 'G%' OR Country = 'Norway';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE City = 'Berlin' OR CustomerName LIKE 'G%' OR Country = 'Norway';

RESULT:
(29, 'Galería del gastrónomo ', 'Eduardo Saavedra ', 'Rambla de Cataluña, 23 ', 'Barcelona ', '08022 ', 'Spain')
(30, 'Godos Cocina Típica ', 'José Pedro Freyre ', 'C/ Romero, 33 ', 'Sevilla ', '41101 ', 'Spain')
(31, 'Gourmet Lanchonetes ', 'André Fonseca ', 'Av. Brasil, 442 ', 'Campinas ', '04876-786 ', 'Brazil')
(32, 'Great Lakes Food Market ', 'Howard Snyder ', '2732 Baker Blvd. ', 'Eugene ', '97403 ', 'USA')
(33, 'GROSELLA-Restaurante ', 'Manuel Pereira ', '5ª Ave. Los Palos Grandes ', 'Caracas ', '1081 ', 'Venezuela')



In [22]:
query ="""
SELECT * FROM Customers
WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');

RESULT:
(29, 'Galería del gastrónomo ', 'Eduardo Saavedra ', 'Rambla de Cataluña, 23 ', 'Barcelona ', '08022 ', 'Spain')
(30, 'Godos Cocina Típica ', 'José Pedro Freyre ', 'C/ Romero, 33 ', 'Sevilla ', '41101 ', 'Spain')
(69, 'Romero y tomillo ', 'Alejandra Camino ', 'Gran Vía, 1 ', 'Madrid ', '28001 ', 'Spain')



In [23]:
query ="""
SELECT * FROM Customers
WHERE NOT Country = 'Spain';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE NOT Country = 'Spain';

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [24]:
query ="""
SELECT * FROM Customers
WHERE CustomerName NOT LIKE 'A%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName NOT LIKE 'A%';

RESULT:
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')
(6, 'Blauer See Delikatessen ', 'Hanna Moos ', 'Forsterstr. 57 ', 'Mannheim ', '68306 ', 'Germany')
(7, 'Blondel père et fils ', 'Frédérique Citeaux ', '24, place Kléber ', 'Strasbourg ', '67000 ', 'France')
(8, 'Bólido Comidas preparadas ', 'Martín Sommer ', 'C/ Araquil, 67 ', 'Madrid ', '28023 ', 'Spain')
(9, "Bon app' ", 'Laurence Lebihans ', '12, rue des Bouchers ', 'Marseille ', '13008 ', 'France')



In [25]:
query ="""
SELECT * FROM Customers
WHERE CustomerID NOT BETWEEN 10 AND 60;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerID NOT BETWEEN 10 AND 60;

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [26]:
query ="""
SELECT * FROM Customers
WHERE City NOT IN ('Paris', 'London');
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE City NOT IN ('Paris', 'London');

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [27]:
query ="""
SELECT * FROM Customers
WHERE NOT CustomerID > 50;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE NOT CustomerID > 50;

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [28]:
query ="""
SELECT * FROM Customers
WHERE NOT CustomerId < 50;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE NOT CustomerId < 50;

RESULT:
(50, 'Maison Dewey ', 'Catherine Dewey ', 'Rue Joseph-Bens 532 ', 'Bruxelles ', 'B-1180 ', 'Belgium')
(51, 'Mère Paillarde ', 'Jean Fresnière ', '43 rue St. Laurent ', 'Montréal ', 'H1J 1C3 ', 'Canada')
(52, 'Morgenstern Gesundkost ', 'Alexander Feuer ', 'Heerstr. 22 ', 'Leipzig ', '04179 ', 'Germany')
(53, 'North/South ', 'Simon Crowther ', 'South House 300 Queensbridge ', 'London ', 'SW7 1RZ ', 'UK')
(54, 'Océano Atlántico Ltda. ', 'Yvonne Moncada ', 'Ing. Gustavo Moncada 8585 Piso 20-A ', 'Buenos Aires ', '1010 ', 'Argentina')



## 3. INSERT

In [29]:
query ="""
INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');
"""
execute(query)

query = """
SELECT *
FROM Customers
WHERE CustomerName == 'Cardinal';
"""
execute(query)

--------------------
QUERY: 
INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE CustomerName == 'Cardinal';

RESULT:
(92, 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway')



In [30]:
query ="""
INSERT INTO Customers (CustomerName, City, Country)
VALUES ('Cardinal', 'Stavanger', 'Norway');
"""
execute(query)

query = """
SELECT *
FROM Customers
WHERE CustomerName == 'Cardinal';
"""
execute(query)

--------------------
QUERY: 
INSERT INTO Customers (CustomerName, City, Country)
VALUES ('Cardinal', 'Stavanger', 'Norway');

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE CustomerName == 'Cardinal';

RESULT:
(92, 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway')
(93, 'Cardinal', None, None, 'Stavanger', None, 'Norway')



In [31]:
query ="""
INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
VALUES
('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway'),
('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'),
('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');
"""
execute(query)


query = """
SELECT *
FROM Customers
WHERE CustomerName IN ('Cardinal', 'Greasy Burger', 'Tasty Tee');
"""
execute(query)

--------------------
QUERY: 
INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
VALUES
('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway'),
('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'),
('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE CustomerName IN ('Cardinal', 'Greasy Burger', 'Tasty Tee');

RESULT:
(92, 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway')
(93, 'Cardinal', None, None, 'Stavanger', None, 'Norway')
(94, 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway')
(95, 'Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway')
(96, 'Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK')



## 4. NULL

In [32]:
query ="""
SELECT CustomerName, ContactName, Address
FROM Customers
WHERE Address IS NULL;
"""
execute(query)

--------------------
QUERY: 
SELECT CustomerName, ContactName, Address
FROM Customers
WHERE Address IS NULL;

RESULT:
('Cardinal', None, None)



In [33]:
query ="""
SELECT CustomerName, ContactName, Address
FROM Customers
WHERE Address IS NOT NULL;
"""
execute(query)

--------------------
QUERY: 
SELECT CustomerName, ContactName, Address
FROM Customers
WHERE Address IS NOT NULL;

RESULT:
('Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ')
('Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ')
('Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ')
('Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ')
('Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ')



In [34]:
query =""" 
select *
from products"""
execute(query)

--------------------
QUERY:  
select *
from products
RESULT:
(1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0)
(2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0)
(4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0)
(5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35)



## 5. Update, Delete, Drop

In [35]:
query ="""
UPDATE Customers
SET ContactName = 'Alfred Schmidt', City= 'Frankfurt'
WHERE CustomerID = 1;
"""
execute(query)

query ="""
SELECT *
FROM Customers
WHERE CustomerID = 1;
"""
execute(query)

--------------------
QUERY: 
UPDATE Customers
SET ContactName = 'Alfred Schmidt', City= 'Frankfurt'
WHERE CustomerID = 1;

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE CustomerID = 1;

RESULT:
(1, 'Alfreds Futterkiste ', 'Alfred Schmidt', 'Obere Str. 57 ', 'Frankfurt', '12209 ', 'Germany')



In [36]:
query ="""
UPDATE Customers
SET ContactName='Juan'
WHERE Country='Mexico';
"""
execute(query)

query ="""
SELECT *
FROM Customers
WHERE Country='Mexico';
"""
execute(query)

--------------------
QUERY: 
UPDATE Customers
SET ContactName='Juan'
WHERE Country='Mexico';

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE Country='Mexico';

RESULT:
(2, 'Ana Trujillo Emparedados y helados ', 'Juan', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Juan', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(13, 'Centro comercial Moctezuma ', 'Juan', 'Sierras de Granada 9993 ', 'México D.F. ', '05022 ', 'Mexico')
(58, 'Pericles Comidas clásicas ', 'Juan', 'Calle Dr. Jorge Cash 321 ', 'México D.F. ', '05033 ', 'Mexico')
(80, 'Tortuga Restaurante ', 'Juan', 'Avda. Azteca 123 ', 'México D.F. ', '05033 ', 'Mexico')



In [37]:
query ="""
UPDATE Customers
SET ContactName='Juan';
"""
execute(query)

query ="""
SELECT *
FROM Customers
"""
execute(query)

--------------------
QUERY: 
UPDATE Customers
SET ContactName='Juan';

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers

RESULT:
(1, 'Alfreds Futterkiste ', 'Juan', 'Obere Str. 57 ', 'Frankfurt', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Juan', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Juan', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Juan', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Juan', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [38]:
query ="""
DELETE 
FROM Customers 
WHERE CustomerName='Alfreds Futterkiste';
"""
execute(query)

query ="""
SELECT *
FROM Customers
WHERE CustomerName='Alfreds Futterkiste';
"""
execute(query)

--------------------
QUERY: 
DELETE 
FROM Customers 
WHERE CustomerName='Alfreds Futterkiste';

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers
WHERE CustomerName='Alfreds Futterkiste';

RESULT:



In [39]:
query ="""
DELETE FROM Customers;
"""
execute(query)

query ="""
SELECT *
FROM Customers
"""
execute(query)

--------------------
QUERY: 
DELETE FROM Customers;

RESULT:

--------------------
QUERY: 
SELECT *
FROM Customers

RESULT:



In [40]:
query ="""
DROP TABLE Customers;
"""
execute(query)

query ="""
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
"""
execute(query)

--------------------
QUERY: 
DROP TABLE Customers;

RESULT:

--------------------
QUERY: 
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

RESULT:
('Products',)
('Categories',)
('Employees',)
('Shippers',)
('Suppliers',)



## 6. Limit in MySQL("Top" in oracle)

In [35]:
query ="""
SELECT * FROM Customers
LIMIT 3;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
LIMIT 3;

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')



In [36]:
query ="""
SELECT * FROM Customers
WHERE Country='Germany'
LIMIT 3;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country='Germany'
LIMIT 3;

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(6, 'Blauer See Delikatessen ', 'Hanna Moos ', 'Forsterstr. 57 ', 'Mannheim ', '68306 ', 'Germany')
(17, 'Drachenblut Delikatessend ', 'Sven Ottlieb ', 'Walserweg 21 ', 'Aachen ', '52066 ', 'Germany')



In [37]:
query ="""
SELECT * FROM Customers
ORDER BY CustomerName DESC
LIMIT 3;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
ORDER BY CustomerName DESC
LIMIT 3;

RESULT:
(91, 'Wolski ', 'Zbyszek ', 'ul. Filtrowa 68 ', 'Walla ', '01-012 ', 'Poland')
(90, 'Wilman Kala ', 'Matti Karttunen ', 'Keskuskatu 45 ', 'Helsinki ', '21240 ', 'Finland')
(89, 'White Clover Markets ', 'Karl Jablonski ', '305 - 14th Ave. S. Suite 3B ', 'Seattle ', '98128 ', 'USA')



## 7. Aggregation

In [38]:
query ="""
SELECT MIN(Price)
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT MIN(Price)
FROM Products;

RESULT:
(2.5,)



In [39]:
query = """
SELECT MIN(Price) AS SmallestPrice
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT MIN(Price) AS SmallestPrice
FROM Products;

RESULT:
(2.5,)



In [40]:
query = """
SELECT MIN(Price) AS SmallestPrice, CategoryID
FROM Products
GROUP BY CategoryID;
"""
execute(query)

--------------------
QUERY: 
SELECT MIN(Price) AS SmallestPrice, CategoryID
FROM Products
GROUP BY CategoryID;

RESULT:
(4.5, 1)
(10.0, 2)
(9.2, 3)
(2.5, 4)
(7.0, 5)



In [41]:
query = """
SELECT COUNT(*)
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(*)
FROM Products;

RESULT:
(77,)



In [42]:
query = """
SELECT COUNT(ProductName)
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(ProductName)
FROM Products;

RESULT:
(77,)



In [43]:
query = """
SELECT COUNT(ProductID)
FROM Products
WHERE Price > 20;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(ProductID)
FROM Products
WHERE Price > 20;

RESULT:
(37,)



In [44]:
query = """
SELECT COUNT(DISTINCT Price)
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(DISTINCT Price)
FROM Products;

RESULT:
(62,)



In [45]:
query = """
SELECT COUNT(*) AS [Number of records]
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(*) AS [Number of records]
FROM Products;

RESULT:
(77,)



In [46]:
query = """
SELECT COUNT(*) AS [Number of records], CategoryID
FROM Products
GROUP BY CategoryID;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(*) AS [Number of records], CategoryID
FROM Products
GROUP BY CategoryID;

RESULT:
(12, 1)
(12, 2)
(13, 3)
(10, 4)
(7, 5)



In [47]:
query = """
SELECT SUM(Quantity)
FROM OrderDetails;
"""
execute(query)

--------------------
QUERY: 
SELECT SUM(Quantity)
FROM OrderDetails;

RESULT:
(12743,)



In [48]:
query = """
SELECT SUM(Quantity)
FROM OrderDetails
WHERE ProductId = 11;
"""
execute(query)

--------------------
QUERY: 
SELECT SUM(Quantity)
FROM OrderDetails
WHERE ProductId = 11;

RESULT:
(182,)



In [49]:
query = """
SELECT SUM(Quantity) AS total
FROM OrderDetails;
"""
execute(query)

--------------------
QUERY: 
SELECT SUM(Quantity) AS total
FROM OrderDetails;

RESULT:
(12743,)



In [50]:
query = """
SELECT OrderID, SUM(Quantity) AS [Total Quantity]
FROM OrderDetails
GROUP BY OrderID;
"""
execute(query)

--------------------
QUERY: 
SELECT OrderID, SUM(Quantity) AS [Total Quantity]
FROM OrderDetails
GROUP BY OrderID;

RESULT:
(10248, 27)
(10249, 49)
(10250, 60)
(10251, 41)
(10252, 105)



In [51]:
query = """
SELECT SUM(Quantity * 10)
FROM OrderDetails;
"""
execute(query)

--------------------
QUERY: 
SELECT SUM(Quantity * 10)
FROM OrderDetails;

RESULT:
(127430,)



In [52]:
query = """
SELECT SUM(Price * Quantity)
FROM OrderDetails
LEFT JOIN Products ON OrderDetails.ProductID = Products.ProductID;
"""
execute(query)

--------------------
QUERY: 
SELECT SUM(Price * Quantity)
FROM OrderDetails
LEFT JOIN Products ON OrderDetails.ProductID = Products.ProductID;

RESULT:
(386424.23,)



In [53]:
query = """
SELECT AVG(Price)
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT AVG(Price)
FROM Products;

RESULT:
(28.866363636363637,)



In [54]:
query = """
SELECT AVG(Price)
FROM Products
WHERE CategoryID = 1;
"""
execute(query)

--------------------
QUERY: 
SELECT AVG(Price)
FROM Products
WHERE CategoryID = 1;

RESULT:
(37.979166666666664,)



In [55]:
query = """
SELECT AVG(Price) AS [average price]
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT AVG(Price) AS [average price]
FROM Products;

RESULT:
(28.866363636363637,)



In [56]:
query = """
SELECT * FROM Products
WHERE price > (SELECT AVG(price) FROM Products);
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE price > (SELECT AVG(price) FROM Products);

RESULT:
(7, "Uncle Bob's Organic Dried Pears", 3, 7, '12 - 1 lb pkgs.', 30.0)
(8, 'Northwoods Cranberry Sauce', 3, 2, '12 - 12 oz jars', 40.0)
(9, 'Mishi Kobe Niku', 4, 6, '18 - 500 g pkgs.', 97.0)
(10, 'Ikura', 4, 8, '12 - 200 ml jars', 31.0)
(12, 'Queso Manchego La Pastora', 5, 4, '10 - 500 g pkgs.', 38.0)



In [57]:
query = """
SELECT AVG(Price) AS AveragePrice, CategoryID
FROM Products
GROUP BY CategoryID;
"""
execute(query)

--------------------
QUERY: 
SELECT AVG(Price) AS AveragePrice, CategoryID
FROM Products
GROUP BY CategoryID;

RESULT:
(37.979166666666664, 1)
(23.0625, 2)
(25.16, 3)
(28.73, 4)
(20.25, 5)



## 8. LIKE

In [58]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE 'a%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE 'a%';

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')



In [59]:
query = """
SELECT * FROM Customers
WHERE city LIKE 'L_nd__';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE city LIKE 'L_nd__';

RESULT:



In [60]:
query = """
SELECT * FROM Customers
WHERE city LIKE '%L%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE city LIKE '%L%';

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')
(9, "Bon app' ", 'Laurence Lebihans ', '12, rue des Bouchers ', 'Marseille ', '13008 ', 'France')
(11, "B's Beverages ", 'Victoria Ashworth ', 'Fauntleroy Circus ', 'London ', 'EC2 5NT ', 'UK')



In [61]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE 'La%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE 'La%';

RESULT:
(40, "La corne d'abondance ", 'Daniel Tonini ', "67, avenue de l'Europe ", 'Versailles ', '78000 ', 'France')
(41, "La maison d'Asie ", 'Annette Roulet ', '1 rue Alsace-Lorraine ', 'Toulouse ', '31000 ', 'France')
(42, 'Laughing Bacchus Wine Cellars ', 'Yoshi Tannamuri ', '1900 Oak St. ', 'Vancouver ', 'V3F 2K1 ', 'Canada')
(43, 'Lazy K Kountry Store ', 'John Steel ', '12 Orchestra Terrace ', 'Walla Walla ', '99362 ', 'USA')



In [62]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE 'a%' OR CustomerName LIKE 'b%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE 'a%' OR CustomerName LIKE 'b%';

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')



In [63]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE '%a';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE '%a';

RESULT:



In [64]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE 'b%s';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE 'b%s';

RESULT:



In [65]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE '%or%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE '%or%';

RESULT:
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(36, 'Hungry Coyote Import Store ', 'Yoshi Latimer ', 'City Center Plaza 516 Main St. ', 'Elgin ', '97827 ', 'USA')
(40, "La corne d'abondance ", 'Daniel Tonini ', "67, avenue de l'Europe ", 'Versailles ', '78000 ', 'France')
(43, 'Lazy K Kountry Store ', 'John Steel ', '12 Orchestra Terrace ', 'Walla Walla ', '99362 ', 'USA')



In [66]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE 'a__%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE 'a__%';

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')



In [67]:
query = """
SELECT * FROM Customers
WHERE CustomerName LIKE '_r%';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerName LIKE '_r%';

RESULT:
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(17, 'Drachenblut Delikatessend ', 'Sven Ottlieb ', 'Walserweg 21 ', 'Aachen ', '52066 ', 'Germany')
(20, 'Ernst Handel ', 'Roland Mendel ', 'Kirchgasse 6 ', 'Graz ', '8010 ', 'Austria')
(25, 'Frankenversand ', 'Peter Franken ', 'Berliner Platz 43 ', 'München ', '80805 ', 'Germany')
(26, 'France restauration ', 'Carine Schmitt ', '54, rue Royale ', 'Nantes ', '44000 ', 'France')



In [68]:
query = """
SELECT * FROM Customers
WHERE Country LIKE 'Spain';
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country LIKE 'Spain';

RESULT:
(8, 'Bólido Comidas preparadas ', 'Martín Sommer ', 'C/ Araquil, 67 ', 'Madrid ', '28023 ', 'Spain')
(22, 'FISSA Fabrica Inter. Salchichas S.A. ', 'Diego Roel ', 'C/ Moralzarzal, 86 ', 'Madrid ', '28034 ', 'Spain')
(29, 'Galería del gastrónomo ', 'Eduardo Saavedra ', 'Rambla de Cataluña, 23 ', 'Barcelona ', '08022 ', 'Spain')
(30, 'Godos Cocina Típica ', 'José Pedro Freyre ', 'C/ Romero, 33 ', 'Sevilla ', '41101 ', 'Spain')
(69, 'Romero y tomillo ', 'Alejandra Camino ', 'Gran Vía, 1 ', 'Madrid ', '28001 ', 'Spain')



## 9. BETWEEN 

In [69]:
query = """
SELECT * FROM Customers
WHERE Country IN ('Germany', 'France', 'UK');
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country IN ('Germany', 'France', 'UK');

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(6, 'Blauer See Delikatessen ', 'Hanna Moos ', 'Forsterstr. 57 ', 'Mannheim ', '68306 ', 'Germany')
(7, 'Blondel père et fils ', 'Frédérique Citeaux ', '24, place Kléber ', 'Strasbourg ', '67000 ', 'France')
(9, "Bon app' ", 'Laurence Lebihans ', '12, rue des Bouchers ', 'Marseille ', '13008 ', 'France')



In [70]:
query = """
SELECT * FROM Customers
WHERE Country NOT IN ('Germany', 'France', 'UK');
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE Country NOT IN ('Germany', 'France', 'UK');

RESULT:
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')
(8, 'Bólido Comidas preparadas ', 'Martín Sommer ', 'C/ Araquil, 67 ', 'Madrid ', '28023 ', 'Spain')
(10, 'Bottom-Dollar Marketse ', 'Elizabeth Lincoln ', '23 Tsawassen Blvd. ', 'Tsawassen ', 'T2F 8M4 ', 'Canada')



In [71]:
query = """
SELECT * FROM Customers
WHERE CustomerID IN (SELECT CustomerID FROM Orders);
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerID IN (SELECT CustomerID FROM Orders);

RESULT:
(2, 'Ana Trujillo Emparedados y helados ', 'Ana Trujillo ', 'Avda. de la Constitución 2222 ', 'México D.F. ', '05021 ', 'Mexico')
(3, 'Antonio Moreno Taquería ', 'Antonio Moreno ', 'Mataderos 2312 ', 'México D.F. ', '05023 ', 'Mexico')
(4, 'Around the Horn ', 'Thomas Hardy ', '120 Hanover Sq. ', 'London ', 'WA1 1DP ', 'UK')
(5, 'Berglunds snabbköp ', 'Christina Berglund ', 'Berguvsvägen 8 ', 'Luleå ', 'S-958 22 ', 'Sweden')
(7, 'Blondel père et fils ', 'Frédérique Citeaux ', '24, place Kléber ', 'Strasbourg ', '67000 ', 'France')



In [72]:
query = """
SELECT * FROM Customers
WHERE CustomerID NOT IN (SELECT CustomerID FROM Orders);
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Customers
WHERE CustomerID NOT IN (SELECT CustomerID FROM Orders);

RESULT:
(1, 'Alfreds Futterkiste ', 'Maria Anders ', 'Obere Str. 57 ', 'Berlin ', '12209 ', 'Germany')
(6, 'Blauer See Delikatessen ', 'Hanna Moos ', 'Forsterstr. 57 ', 'Mannheim ', '68306 ', 'Germany')
(12, 'Cactus Comidas para llevar ', 'Patricio Simpson ', 'Cerrito 333 ', 'Buenos Aires ', '1010 ', 'Argentina')
(22, 'FISSA Fabrica Inter. Salchichas S.A. ', 'Diego Roel ', 'C/ Moralzarzal, 86 ', 'Madrid ', '28034 ', 'Spain')
(26, 'France restauration ', 'Carine Schmitt ', '54, rue Royale ', 'Nantes ', '44000 ', 'France')



In [73]:
query = """
SELECT * FROM Products
WHERE Price BETWEEN 10 AND 20;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE Price BETWEEN 10 AND 20;

RESULT:
(1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0)
(2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0)
(15, 'Genen Shouyu', 6, 2, '24 - 250 ml bottles', 15.5)
(16, 'Pavlova', 7, 3, '32 - 500 g boxes', 17.45)



In [74]:
query = """
SELECT * FROM Products
WHERE Price NOT BETWEEN 10 AND 20;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE Price NOT BETWEEN 10 AND 20;

RESULT:
(4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0)
(5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35)
(6, "Grandma's Boysenberry Spread", 3, 2, '12 - 8 oz jars', 25.0)
(7, "Uncle Bob's Organic Dried Pears", 3, 7, '12 - 1 lb pkgs.', 30.0)
(8, 'Northwoods Cranberry Sauce', 3, 2, '12 - 12 oz jars', 40.0)



In [75]:
query = """
SELECT * FROM Products
WHERE Price BETWEEN 10 AND 20
AND CategoryID IN (1,2,3);
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE Price BETWEEN 10 AND 20
AND CategoryID IN (1,2,3);

RESULT:
(1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0)
(2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0)
(15, 'Genen Shouyu', 6, 2, '24 - 250 ml bottles', 15.5)
(16, 'Pavlova', 7, 3, '32 - 500 g boxes', 17.45)



In [76]:
query = """
SELECT * FROM Products
WHERE ProductName BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
ORDER BY ProductName;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE ProductName BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
ORDER BY ProductName;

RESULT:
(18, 'Carnarvon Tigers', 7, 8, '16 kg pkg.', 62.5)
(1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0)
(2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0)
(39, 'Chartreuse verte', 18, 1, '750 cc per bottle', 18.0)
(4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0)



In [77]:
query = """
SELECT * FROM Products
WHERE ProductName BETWEEN "Carnarvon Tigers" AND "Chef Anton's Cajun Seasoning"
ORDER BY ProductName;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE ProductName BETWEEN "Carnarvon Tigers" AND "Chef Anton's Cajun Seasoning"
ORDER BY ProductName;

RESULT:
(18, 'Carnarvon Tigers', 7, 8, '16 kg pkg.', 62.5)
(1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0)
(2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0)
(39, 'Chartreuse verte', 18, 1, '750 cc per bottle', 18.0)
(4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0)



In [78]:
query = """
SELECT * FROM Products
WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
ORDER BY ProductName;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
ORDER BY ProductName;

RESULT:
(17, 'Alice Mutton', 7, 6, '20 - 1 kg tins', 39.0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0)
(40, 'Boston Crab Meat', 19, 8, '24 - 4 oz tins', 18.4)
(60, 'Camembert Pierrot', 28, 4, '15 - 300 g rounds', 34.0)
(30, 'Nord-Ost Matjeshering', 13, 8, '10 - 200 g glasses', 25.89)



In [79]:
query = """
SELECT * FROM Products
WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
ORDER BY ProductName;
"""
execute(query)

--------------------
QUERY: 
SELECT * FROM Products
WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
ORDER BY ProductName;

RESULT:
(17, 'Alice Mutton', 7, 6, '20 - 1 kg tins', 39.0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0)
(40, 'Boston Crab Meat', 19, 8, '24 - 4 oz tins', 18.4)
(60, 'Camembert Pierrot', 28, 4, '15 - 300 g rounds', 34.0)
(30, 'Nord-Ost Matjeshering', 13, 8, '10 - 200 g glasses', 25.89)



## 10. JOIN

In [80]:
query = """
SELECT CustomerName, Address + ', ' + PostalCode + ' ' + City + ', ' + Country AS Address
FROM Customers;
"""
execute(query)

--------------------
QUERY: 
SELECT CustomerName, Address + ', ' + PostalCode + ' ' + City + ', ' + Country AS Address
FROM Customers;

RESULT:
('Alfreds Futterkiste ', 12209)
('Ana Trujillo Emparedados y helados ', 5021)
('Antonio Moreno Taquería ', 5023)
('Around the Horn ', 120)
('Berglunds snabbköp ', 0)



In [81]:
query = """
SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate
FROM Orders
INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;
"""
execute(query)

--------------------
QUERY: 
SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate
FROM Orders
INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;

RESULT:
(10248, 'Wilman Kala ', '7/4/1996')
(10249, 'Tradição Hipermercados ', '7/5/1996')
(10250, 'Hanari Carnes ', '7/8/1996')
(10251, 'Victuailles en stock ', '7/8/1996')
(10252, 'Suprêmes délices ', '7/9/1996')



In [82]:
query = """
SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
FROM Products
INNER JOIN Categories ON Products.CategoryID = Categories.CategoryID;
"""
execute(query)

--------------------
QUERY: 
SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
FROM Products
INNER JOIN Categories ON Products.CategoryID = Categories.CategoryID;

RESULT:
(1, 'Chais', 'Beverages')
(2, 'Chang', 'Beverages')
(3, 'Aniseed Syrup', 'Condiments')
(4, "Chef Anton's Cajun Seasoning", 'Condiments')
(5, "Chef Anton's Gumbo Mix", 'Condiments')



In [83]:
query = """
SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
FROM Products
JOIN Categories ON Products.CategoryID = Categories.CategoryID;
"""
execute(query)

--------------------
QUERY: 
SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
FROM Products
JOIN Categories ON Products.CategoryID = Categories.CategoryID;

RESULT:
(1, 'Chais', 'Beverages')
(2, 'Chang', 'Beverages')
(3, 'Aniseed Syrup', 'Condiments')
(4, "Chef Anton's Cajun Seasoning", 'Condiments')
(5, "Chef Anton's Gumbo Mix", 'Condiments')



In [84]:
query = """
SELECT Orders.OrderID, Customers.CustomerName, Shippers.ShipperName
FROM ((Orders
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID)
INNER JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID);
"""
execute(query)

--------------------
QUERY: 
SELECT Orders.OrderID, Customers.CustomerName, Shippers.ShipperName
FROM ((Orders
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID)
INNER JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID);

RESULT:
(10248, 'Wilman Kala ', 'Federal Shipping')
(10249, 'Tradição Hipermercados ', 'Speedy Express')
(10250, 'Hanari Carnes ', 'United Package')
(10251, 'Victuailles en stock ', 'Speedy Express')
(10252, 'Suprêmes délices ', 'United Package')



In [85]:
query = """
SELECT Customers.CustomerName, Orders.OrderID
FROM Customers
LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
ORDER BY Customers.CustomerName;
"""
execute(query)

--------------------
QUERY: 
SELECT Customers.CustomerName, Orders.OrderID
FROM Customers
LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
ORDER BY Customers.CustomerName;

RESULT:
('Alfreds Futterkiste ', None)
('Ana Trujillo Emparedados y helados ', 10308)
('Antonio Moreno Taquería ', 10365)
('Around the Horn ', 10355)
('Around the Horn ', 10383)



In [86]:
query = """
SELECT Orders.OrderID, Employees.LastName, Employees.FirstName
FROM Orders
RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
ORDER BY Orders.OrderID;
"""
execute(query)

--------------------
QUERY: 
SELECT Orders.OrderID, Employees.LastName, Employees.FirstName
FROM Orders
RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
ORDER BY Orders.OrderID;

RESULT:
(None, 'West', 'Adam')
(10248, 'Buchanan', 'Steven')
(10249, 'Suyama', 'Michael')
(10250, 'Peacock', 'Margaret')
(10251, 'Leverling', 'Janet')



In [87]:
query = """
SELECT Customers.CustomerName, Orders.OrderID
FROM Customers
FULL OUTER JOIN Orders ON Customers.CustomerID=Orders.CustomerID
ORDER BY Customers.CustomerName;
"""
execute(query)

--------------------
QUERY: 
SELECT Customers.CustomerName, Orders.OrderID
FROM Customers
FULL OUTER JOIN Orders ON Customers.CustomerID=Orders.CustomerID
ORDER BY Customers.CustomerName;

RESULT:
('Alfreds Futterkiste ', None)
('Ana Trujillo Emparedados y helados ', 10308)
('Antonio Moreno Taquería ', 10365)
('Around the Horn ', 10355)
('Around the Horn ', 10383)



In [88]:
query = """
SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
FROM Customers A, Customers B
WHERE A.CustomerID <> B.CustomerID
AND A.City = B.City
ORDER BY A.City;
"""
execute(query)

--------------------
QUERY: 
SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
FROM Customers A, Customers B
WHERE A.CustomerID <> B.CustomerID
AND A.City = B.City
ORDER BY A.City;

RESULT:
('Cactus Comidas para llevar ', 'Océano Atlántico Ltda. ', 'Buenos Aires ')
('Cactus Comidas para llevar ', 'Rancho grande ', 'Buenos Aires ')
('Océano Atlántico Ltda. ', 'Cactus Comidas para llevar ', 'Buenos Aires ')
('Océano Atlántico Ltda. ', 'Rancho grande ', 'Buenos Aires ')
('Rancho grande ', 'Cactus Comidas para llevar ', 'Buenos Aires ')



## 11. UNION

In [89]:
query = """
SELECT City FROM Customers
UNION ALL
SELECT City FROM Suppliers
ORDER BY City;
"""
execute(query)

--------------------
QUERY: 
SELECT City FROM Customers
UNION ALL
SELECT City FROM Suppliers
ORDER BY City;

RESULT:
('Aachen ',)
('Albuquerque ',)
('Anchorage ',)
('Ann Arbor',)
('Annecy',)



In [90]:
query = """
SELECT City, Country FROM Customers
WHERE Country='Germany'
UNION
SELECT City, Country FROM Suppliers
WHERE Country='Germany'
ORDER BY City;
"""
execute(query)

--------------------
QUERY: 
SELECT City, Country FROM Customers
WHERE Country='Germany'
UNION
SELECT City, Country FROM Suppliers
WHERE Country='Germany'
ORDER BY City;

RESULT:
('Aachen ', 'Germany')
('Berlin', 'Germany')
('Berlin ', 'Germany')
('Brandenburg ', 'Germany')
('Cunewalde ', 'Germany')



In [91]:
query = """
SELECT City, Country FROM Customers
WHERE Country='Germany'
UNION ALL
SELECT City, Country FROM Suppliers
WHERE Country='Germany'
ORDER BY City;
"""
execute(query)

--------------------
QUERY: 
SELECT City, Country FROM Customers
WHERE Country='Germany'
UNION ALL
SELECT City, Country FROM Suppliers
WHERE Country='Germany'
ORDER BY City;

RESULT:
('Aachen ', 'Germany')
('Berlin', 'Germany')
('Berlin ', 'Germany')
('Brandenburg ', 'Germany')
('Cunewalde ', 'Germany')



In [92]:
query = """
SELECT 'Customer' AS Type, ContactName, City, Country
FROM Customers
UNION
SELECT 'Supplier', ContactName, City, Country
FROM Suppliers;
"""
execute(query)

--------------------
QUERY: 
SELECT 'Customer' AS Type, ContactName, City, Country
FROM Customers
UNION
SELECT 'Supplier', ContactName, City, Country
FROM Suppliers;

RESULT:
('Customer', None, 'Stavanger', 'Norway')
('Customer', 'Alejandra Camino ', 'Madrid ', 'Spain')
('Customer', 'Alexander Feuer ', 'Leipzig ', 'Germany')
('Customer', 'Ana Trujillo ', 'México D.F. ', 'Mexico')
('Customer', 'Anabela Domingues ', 'São Paulo ', 'Brazil')



## 12. GROUP BY, HAVING

In [93]:
query = """
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country;

RESULT:
(3, 'Argentina')
(2, 'Austria')
(2, 'Belgium')
(9, 'Brazil')
(3, 'Canada')



In [94]:
query = """
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;

RESULT:
(13, 'USA')
(11, 'Germany')
(11, 'France')
(9, 'Brazil')
(8, 'UK')



In [95]:
query = """
SELECT Shippers.ShipperName, COUNT(Orders.OrderID) AS NumberOfOrders FROM Orders
LEFT JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID
GROUP BY ShipperName;
"""
execute(query)

--------------------
QUERY: 
SELECT Shippers.ShipperName, COUNT(Orders.OrderID) AS NumberOfOrders FROM Orders
LEFT JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID
GROUP BY ShipperName;

RESULT:
('Federal Shipping', 68)
('Speedy Express', 54)
('United Package', 74)



In [96]:
query = """
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
HAVING COUNT(CustomerID) > 5;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
HAVING COUNT(CustomerID) > 5;

RESULT:
(9, 'Brazil')
(11, 'France')
(11, 'Germany')
(8, 'UK')
(13, 'USA')



In [97]:
query = """
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
HAVING COUNT(CustomerID) > 5
ORDER BY COUNT(CustomerID) DESC;
"""
execute(query)

--------------------
QUERY: 
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
HAVING COUNT(CustomerID) > 5
ORDER BY COUNT(CustomerID) DESC;

RESULT:
(13, 'USA')
(11, 'Germany')
(11, 'France')
(9, 'Brazil')
(8, 'UK')



In [98]:
query = """
SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
FROM (Orders
INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID)
GROUP BY LastName
HAVING COUNT(Orders.OrderID) > 10;
"""
execute(query)

--------------------
QUERY: 
SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
FROM (Orders
INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID)
GROUP BY LastName
HAVING COUNT(Orders.OrderID) > 10;

RESULT:
('Buchanan', 11)
('Callahan', 27)
('Davolio', 29)
('Fuller', 20)
('King', 14)



In [99]:
query = """
SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
FROM Orders
INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
WHERE LastName = 'Davolio' OR LastName = 'Fuller'
GROUP BY LastName
HAVING COUNT(Orders.OrderID) > 25;
"""
execute(query)

--------------------
QUERY: 
SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
FROM Orders
INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
WHERE LastName = 'Davolio' OR LastName = 'Fuller'
GROUP BY LastName
HAVING COUNT(Orders.OrderID) > 25;

RESULT:
('Davolio', 29)



## 13. EXIST, ANY, ALL

In [100]:
query = """
SELECT SupplierName
FROM Suppliers
WHERE EXISTS (SELECT 1 FROM Products WHERE Products.SupplierID = Suppliers.SupplierID AND Price < 20);
"""
execute(query)

--------------------
QUERY: 
SELECT SupplierName
FROM Suppliers
WHERE EXISTS (SELECT 1 FROM Products WHERE Products.SupplierID = Suppliers.SupplierID AND Price < 20);

RESULT:
('Exotic Liquid',)
('New Orleans Cajun Delights',)
('Tokyo Traders',)
("Mayumi's",)
('Pavlova, Ltd.',)



In [101]:
query = """
SELECT SupplierName
FROM Suppliers
WHERE EXISTS (SELECT 1 FROM Products WHERE Products.SupplierID = Suppliers.SupplierID AND Price = 22);
"""
execute(query)

--------------------
QUERY: 
SELECT SupplierName
FROM Suppliers
WHERE EXISTS (SELECT 1 FROM Products WHERE Products.SupplierID = Suppliers.SupplierID AND Price = 22);

RESULT:
('New Orleans Cajun Delights',)



In [102]:
query = """
SELECT ProductName
FROM Products
WHERE ProductID IN (SELECT ProductID FROM OrderDetails WHERE Quantity = 10);
"""
execute(query)

--------------------
QUERY: 
SELECT ProductName
FROM Products
WHERE ProductID IN (SELECT ProductID FROM OrderDetails WHERE Quantity = 10);

RESULT:
('Chais',)
('Chang',)
("Chef Anton's Cajun Seasoning",)
("Uncle Bob's Organic Dried Pears",)
('Konbu',)



In [103]:
query = """
SELECT ProductName
FROM Products
WHERE ProductID IN (SELECT ProductID FROM OrderDetails WHERE Quantity > 99);
"""
execute(query)

--------------------
QUERY: 
SELECT ProductName
FROM Products
WHERE ProductID IN (SELECT ProductID FROM OrderDetails WHERE Quantity > 99);

RESULT:
('Steeleye Stout',)
('Pâté chinois',)



In [104]:
query = """
SELECT ProductName
FROM Products
WHERE ProductID IN (SELECT ProductID FROM OrderDetails WHERE Quantity > 1000);
"""
execute(query)

--------------------
QUERY: 
SELECT ProductName
FROM Products
WHERE ProductID IN (SELECT ProductID FROM OrderDetails WHERE Quantity > 1000);

RESULT:



In [105]:
query = """
SELECT ProductName
FROM Products;
"""
execute(query)

--------------------
QUERY: 
SELECT ProductName
FROM Products;

RESULT:
('Chais',)
('Chang',)
('Aniseed Syrup',)
("Chef Anton's Cajun Seasoning",)
("Chef Anton's Gumbo Mix",)



In [106]:
query = """
SELECT ProductName
FROM Products
WHERE ProductID = (SELECT ProductID FROM OrderDetails WHERE Quantity = 10 LIMIT 1);
"""
execute(query)

--------------------
QUERY: 
SELECT ProductName
FROM Products
WHERE ProductID = (SELECT ProductID FROM OrderDetails WHERE Quantity = 10 LIMIT 1);

RESULT:
('Singaporean Hokkien Fried Mee',)



## 14. SELECT + INSERT

In [107]:
query = """
INSERT INTO Customers (CustomerName, City, Country)
SELECT SupplierName, City, Country FROM Suppliers;
"""

execute(query)

--------------------
QUERY: 
INSERT INTO Customers (CustomerName, City, Country)
SELECT SupplierName, City, Country FROM Suppliers;

RESULT:



In [108]:
query = """
INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
SELECT SupplierName, ContactName, Address, City, PostalCode, Country FROM Suppliers;
"""

execute(query)

--------------------
QUERY: 
INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
SELECT SupplierName, ContactName, Address, City, PostalCode, Country FROM Suppliers;

RESULT:



In [109]:
query = """
INSERT INTO Customers (CustomerName, City, Country)
SELECT SupplierName, City, Country FROM Suppliers
WHERE Country='Germany';
"""

execute(query)

--------------------
QUERY: 
INSERT INTO Customers (CustomerName, City, Country)
SELECT SupplierName, City, Country FROM Suppliers
WHERE Country='Germany';

RESULT:



## 15. CASE

In [110]:
query = """
SELECT OrderID, Quantity,
CASE
    WHEN Quantity > 30 THEN 'The quantity is greater than 30'
    WHEN Quantity = 30 THEN 'The quantity is 30'
    ELSE 'The quantity is under 30'
END AS QuantityText
FROM OrderDetails;
"""

execute(query)

--------------------
QUERY: 
SELECT OrderID, Quantity,
CASE
    WHEN Quantity > 30 THEN 'The quantity is greater than 30'
    WHEN Quantity = 30 THEN 'The quantity is 30'
    ELSE 'The quantity is under 30'
END AS QuantityText
FROM OrderDetails;

RESULT:
(10248, 12, 'The quantity is under 30')
(10248, 10, 'The quantity is under 30')
(10248, 5, 'The quantity is under 30')
(10249, 9, 'The quantity is under 30')
(10249, 40, 'The quantity is greater than 30')



In [111]:
query = """
SELECT CustomerName, City, Country
FROM Customers
ORDER BY
(CASE
    WHEN City IS NULL THEN Country
    ELSE City
END);
"""

execute(query)

--------------------
QUERY: 
SELECT CustomerName, City, Country
FROM Customers
ORDER BY
(CASE
    WHEN City IS NULL THEN Country
    ELSE City
END);

RESULT:
('Drachenblut Delikatessend ', 'Aachen ', 'Germany')
('Rattlesnake Canyon Grocery ', 'Albuquerque ', 'USA')
('Old World Delicatessen ', 'Anchorage ', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')

